# HW 1 By Steven Jia

## 1. Github page

My GitHub page can be found here: https://github.com/SpuriusSergius/DATA441

## 2. Ridge Regression

In [1]:
import torch
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
class Ridge:
    def __init__(self, rate = 0.01, epochs = 10000, alpha = 0.5):
      self.rate = rate
      self.epochs = epochs
      self.alpha = alpha
      self.scaler = StandardScaler()

    def predict(self, x_new):
      return torch.tensor(self.scaler.transform(x_new))@self.weights + self.intercept

    def fit(self, x, y):

      Xtensor = torch.tensor(self.scaler.fit_transform(x))
      Ytensor = torch.tensor(y)
      obs, feat = Xtensor.size()


      self.weights = torch.zeros(feat, 1, dtype=torch.double)
      self.intercept = 0

      for round in range(self.epochs):
        predict = Xtensor@self.weights + self.intercept

        # cost = (predict - Ytensor)**2 + self.alpha * torch.sum(self.weights**2)
        db = torch.mean(predict - Ytensor[:, None])
        dw = (Xtensor.T@(predict - Ytensor[:, None]) + self.alpha * torch.sum(self.weights**2)) / Xtensor.size(0)

        self.weights = self.weights - self.rate*dw
        self.intercept = self.intercept - self.rate*db

      return self.weights

#### Testing My Implmentation

In [3]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/DATA 441/cars.csv')

In [4]:
data

,MPG,CYL,ENG,WGT
0,18.0,8,307.0,3504
1,15.0,8,350.0,3693
2,18.0,8,318.0,3436
3,16.0,8,304.0,3433
4,17.0,8,302.0,3449
...,...,...,...,...
387,27.0,4,140.0,2790
388,44.0,4,97.0,2130
389,32.0,4,135.0,2295
390,28.0,4,120.0,2625


In [5]:
x = data.drop(columns=['MPG']).values
y = data['MPG'].values

In [6]:
model = Ridge()

In [7]:
model.fit(x, y)

tensor([[-0.4664],
        [-1.3359],
        [-4.8550]], dtype=torch.float64)

In [9]:
torch.mean((model.predict(x).flatten() - y)**2)

tensor(18.2729, dtype=torch.float64)

## 3. Compelete the exercise in Application to Locally Weighted Regression

#### Needed imports and functions from other notebook

In [42]:
import numpy as np
from sklearn import linear_model
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse

In [43]:
# Gaussian Kernel
def Gaussian(x):
  return np.where(np.abs(x)>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*x**2))

In [44]:
# this is the correct vectorized version
def tricubic(x):
  return np.where(np.abs(x)>1,0,(1-np.abs(x)**3)**3)

In [45]:
# Epanechnikov Kernel
def Epanechnikov(x):
  return np.where(np.abs(x)>1,0,3/4*(1-np.abs(x)**2))

In [46]:
# Quartic Kernel
def Quartic(x):
  return np.where(np.abs(x)>1,0,15/16*(1-np.abs(x)**2)**2)

In [47]:
# how about when both u and v are matrices?
def dist(u, v):
  D = []
  if len(v.shape) == 1:
      v = v.reshape(1, -1)
  # we would like all the pairwise combinations if u and v are matrices
  # we could avoid two for loops if we consider broadcasting
  for rowi in v:
    D.append(np.sqrt(np.sum((u-rowi)**2, axis = 1)))
  return np.array(D)

In [48]:
def weight_function(u,v,kern = Gaussian, tau = 0.5):
    return kern(dist(u,v)/(2*tau))

#### Fixed Lowess Class

In [49]:
class Lowess:
    def __init__(self, kernel = Gaussian, tau=0.05):
        self.kernel = kernel
        self.tau = tau

    def fit(self, x, y):
        kernel = self.kernel
        tau = self.tau
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        lm = linear_model.Ridge(alpha=0.001)
        w = weight_function(x,x_new,self.kernel,self.tau)

        if np.isscalar(x_new):
          lm.fit(np.diag(w).dot(x.reshape(-1,1)),np.diag(w).dot(y.reshape(-1,1)))
          yest = lm.predict([[x_new]])[0][0]
        else:
          n = len(x_new)
          yest_test = np.zeros(n)
          #Looping through all x-points
          for i in range(n):
            lm.fit(np.diag(w[i,:])@x,np.diag(w[i,:])@y)
            yest_test[i] = lm.predict(x_new[i].reshape(1, -1))
        return yest_test

#### Testing

In [50]:
q3x = data.drop(columns=['MPG']).values
q3y = data['MPG'].values

In [51]:
model = Lowess(kernel = Epanechnikov, tau = 0.15)

In [52]:
scale = MinMaxScaler()
xscaled = scale.fit_transform(q3x)

In [53]:
model.fit(xscaled, q3y)

In [54]:
yhat = model.predict(xscaled)

In [55]:
mse(yhat, y)

15.150770647058264